In [1]:
import sqlite3
import pandas as pd
import numpy as np

#Configurar pandas para mostrar exibir todas as colunas
pd.set_option('display.max_columns', None)

In [2]:
#Ler arquivo CSV usando codificação válida, pulando linhas quebradas e definindo ";" como separador
df = pd.read_csv('C:/Users/Gustavo/Documents/Github/BI/MICRODADOS.csv', encoding='ISO-8859-1', on_bad_lines='skip', sep=';')

In [3]:
#Selecionar registros de pessoas que morreram
dfFato = df[df['DataObito'].notna()].copy(deep=True)

#Selecionar apenas mortes por covid
dfFato = dfFato[dfFato['Evolucao'] == 'Óbito pelo COVID-19']

#Liberar memória do dataframe original do csv
del df

#Resetar índice do dataframe dfFato
dfFato = dfFato.reset_index(drop=True)

In [4]:
########### Cria tabela fato "tabela_fato" no SQLite carregando a partir do dataframe ###########

# Criar a conexão com o banco de dados
conn = sqlite3.connect('bancodedados.db')

# Carregar o dataframe para o banco de dados como uma tabela fato
dfFato.to_sql('tabela_fato', conn, if_exists='replace', index=False)

# Fecha a conexão com o banco de dados
conn.close()

In [5]:
########### Cria tabela dimensao_localidade ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

#Apaga a tabela dimensao_localidade se ela já existe
cursor.execute('DROP TABLE IF EXISTS dimensao_localidade')

#Cria a tabela dimensao_localidade
cursor.execute('''
    CREATE TABLE dimensao_localidade (
        ID INTEGER PRIMARY KEY,
        Municipio TEXT,
        Bairro TEXT
    )
''')

#Insere os dados na tabela, usando o índice da tabela fato como chave primária
for i, row in dfFato.iterrows():
    query = f"""
        INSERT INTO dimensao_localidade (ID, Municipio, Bairro)
        VALUES ({i}, "{row['Municipio']}", "{row['Bairro']}")
    """
    cursor.execute(query)


#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()

In [6]:
########### Cria tabela dimensao_caracteristicas_individuo ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

#Apaga a tabela dimensao_caracteristicas_individuo se ela já existe
cursor.execute('DROP TABLE IF EXISTS dimensao_caracteristicas_individuo')

#Cria a tabela dimensao_caracteristicas_individuo
cursor.execute('''
    CREATE TABLE dimensao_caracteristicas_individuo (
        ID INTEGER PRIMARY KEY,
        FaixaEtaria TEXT,
        IdadeNaDataNotificacao TEXT,
        Sexo TEXT,
        RacaCor TEXT,
        Escolaridade TEXT,
        Gestante TEXT
    )
''')

#Insere os dados na tabela, usando o índice da tabela fato como chave primária
for i, row in dfFato.iterrows():
    query = f"""
        INSERT INTO dimensao_caracteristicas_individuo (ID, FaixaEtaria, IdadeNaDataNotificacao, Sexo, RacaCor, Escolaridade, Gestante)
        VALUES ({i}, "{row['FaixaEtaria']}", "{row['IdadeNaDataNotificacao']}", "{row['Sexo']}", "{row['RacaCor']}", "{row['Escolaridade']}", "{row['Gestante']}")
    """
    cursor.execute(query)


#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()

In [7]:
########### Cria tabela dimensao_sintomas ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

#Apaga a tabela dimensao_sintomas se ela já existe
cursor.execute('DROP TABLE IF EXISTS dimensao_sintomas')

#Cria a tabela dimensao_sintomas
cursor.execute('''
    CREATE TABLE dimensao_sintomas (
        ID INTEGER PRIMARY KEY,
        Febre TEXT,
        DificuldadeRespiratoria TEXT,
        Tosse TEXT,
        Coriza TEXT,
        DorGarganta TEXT,
        Diarreia TEXT,
        Cefaleia TEXT
    )
''')

#Insere os dados na tabela, usando o índice da tabela fato como chave primária
for i, row in dfFato.iterrows():
    query = f"""
        INSERT INTO dimensao_sintomas (ID, Febre, DificuldadeRespiratoria, Tosse, Coriza, DorGarganta, Diarreia, Cefaleia)
        VALUES ({i}, "{row['Febre']}", "{row['DificuldadeRespiratoria']}", "{row['Tosse']}", "{row['Coriza']}", "{row['DorGarganta']}", "{row['Diarreia']}", "{row['Cefaleia']}")
    """
    cursor.execute(query)


#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()

In [8]:
########### Cria tabela dimensao_comorbidades ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

#Apaga a tabela dimensao_comorbidades se ela já existe
cursor.execute('DROP TABLE IF EXISTS dimensao_comorbidades')

#Cria a tabela dimensao_comorbidades
cursor.execute('''
    CREATE TABLE dimensao_comorbidades (
        ID INTEGER PRIMARY KEY,
        ComorbidadePulmao TEXT,
        ComorbidadeCardio TEXT,
        ComorbidadeRenal TEXT,
        ComorbidadeDiabetes TEXT,
        ComorbidadeTabagismo TEXT,
        ComorbidadeObesidade TEXT
    )
''')

#Insere os dados na tabela, usando o índice da tabela fato como chave primária
for i, row in dfFato.iterrows():
    query = f"""
        INSERT INTO dimensao_comorbidades (ID, ComorbidadePulmao, ComorbidadeCardio, ComorbidadeRenal, ComorbidadeDiabetes, ComorbidadeTabagismo, ComorbidadeObesidade)
        VALUES ({i}, "{row['ComorbidadePulmao']}", "{row['ComorbidadeCardio']}", "{row['ComorbidadeRenal']}", "{row['ComorbidadeDiabetes']}", "{row['ComorbidadeTabagismo']}", "{row['ComorbidadeObesidade']}")
    """
    cursor.execute(query)


#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()

In [9]:
########### Visualizar tabela criada ###########
# Criar a conexão com o banco de dados
conn = sqlite3.connect('bancodedados.db')

# Executar a consulta SQL para selecionar todos os dados da tabela (Ex.: dimensao_comorbidades)
consultaTabela = pd.read_sql_query("SELECT * from dimensao_comorbidades", conn)

# Fechar a conexão com o banco de dados
conn.close()

# Visualizar o dataframe
consultaTabela

,ID,ComorbidadePulmao,ComorbidadeCardio,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade
0,0,Não,Não,Não,Não,Não,Não
1,1,Não,Não,Não,Não,Não,Não
2,2,Não,Não,Não,Não,Não,Não
3,3,Não,Sim,Não,Não,Não,Não
4,4,Não,Não,Não,Não,Não,Não
...,...,...,...,...,...,...,...
15067,15067,Não,Sim,Não,Não,Não,Não
15068,15068,-,-,-,-,-,Sim
15069,15069,Não,Sim,Sim,Sim,Não,Não
15070,15070,-,-,-,-,-,-
